# Scrape data from the Internet to create dataset

In [1]:
from bs4 import BeautifulSoup
import cloudscraper
import re
import timeit
import pandas as pd
from konoha import SentenceTokenizer

# a popular website to practice JLPT exams, contains many reading exercises and listening transcript
WEBSITE = "https://japanesetest4you.com/"

In [2]:
# to scrape website hosted by cloudflare
SCRAPER = cloudscraper.create_scraper()

def request_html(url) -> BeautifulSoup:
    response = SCRAPER.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

### Scrape links to reading exercise menu from homepage

In [3]:
def extract_menu_urls(soup: BeautifulSoup):
    """
    Input:
    -   soup: homepage https://japanesetest4you.com/

    Output:
    -   [
            'https://japanesetest4you.com/category/jlpt-n1/jlpt-n1-reading-test/',
            'https://japanesetest4you.com/category/jlpt-n2/jlpt-n2-reading-test/',
        ]
    """
    links_tag =soup.find_all("a", string=re.compile(r".*READING TEST.*"))
    links_to_menu = sorted([l.attrs["href"] for l in links_tag])
    return links_to_menu

**usage example**

In [4]:
soup = request_html(WEBSITE)

In [5]:
links_to_menu = extract_menu_urls(soup)
links_to_menu

['https://japanesetest4you.com/category/jlpt-n1/jlpt-n1-reading-test/',
 'https://japanesetest4you.com/category/jlpt-n2/jlpt-n2-reading-test/',
 'https://japanesetest4you.com/category/jlpt-n3/jlpt-n3-reading-test/',
 'https://japanesetest4you.com/category/jlpt-n4/jlpt-n4-reading-test/',
 'https://japanesetest4you.com/category/jlpt-n5/jlpt-n5-reading-tests/']

### Scrape links to reading exercise from menu

In [6]:
def extract_test_urls(soup: BeautifulSoup):
    """
    Input:
    -   soup: from reading test menu in each jlpt level

    Output:
    -   [
            'https://japanesetest4you.com/japanese-language-proficiency-test-jlpt-n3-reading-exercise-1/#more-703',
            'https://japanesetest4you.com/japanese-language-proficiency-test-jlpt-n3-reading-exercise-2/#more-705',
        ]
    """
    links_div =soup.find_all("div", class_="readmore")
    links_to_exercise = [l.a.attrs["href"] for l in links_div]
    return links_to_exercise

**usage example**

In [7]:
soup = request_html(r"https://japanesetest4you.com/category/jlpt-n3/jlpt-n3-reading-test/")

In [8]:
links_to_exercise = extract_test_urls(soup)
links_to_exercise[:3]

['https://japanesetest4you.com/japanese-language-proficiency-test-jlpt-n3-reading-exercise-1/#more-703',
 'https://japanesetest4you.com/japanese-language-proficiency-test-jlpt-n3-reading-exercise-2/#more-705',
 'https://japanesetest4you.com/japanese-language-proficiency-test-jlpt-n3-reading-exercise-3/#more-708']

### Scrape reading exercise given url

In [9]:
def extract_reading_exercise(soup: BeautifulSoup):
    """
    Input:
    -   soup: from each reading exercise html

    Output:
    -   {
            'Reading Passage 1': [
                <p>学生: すみません。この本をかりたいです。<br/>としょかんの人: この学校の学生ですか。,
                <p>1. [_______] には何を入れますか。<br/><input name="quest1" type="radio" value="1"/>,
            ],
            'Reading Passage 2': [
                <p>リン「むらたさん、ちょっといいですか。」<br/>むらた「はい、なんですか。」,
                <p>5. ただしいものはどれですか。<br/><input name="quest5" type="radio" value="1"/>,
            ],
            'Answer Key:': [
                <p>Question 1: 1<br/>Question 2: 2<br/>Question 3: 4<br/>Question 4: 3<br/>Question 5: 4</p>,
            ],
            'JLPT N5 Kanji Lesson #3: 中 (chuu)': [
                <p><span>Download the video lesson and transcript</a></span></p>,
            ],
            'New words:': [
                <p>学生 (gakusei): student</p>,
                <p><span>Download the video lesson and transcript</span></p>,
            ],
        }
    """
    # get the body div
    passage_div = soup.find_all("div",class_= "entry clearfix")[0]
    content_nodes = [p for p in passage_div.children if p.name == 'p']

    # get the heading nodes
    passages_heading_nodes = [n for n in content_nodes if n.strong]

    # classify the texts according to heading
    sections = {}
    cur_heading = None
    for n in content_nodes:
        # make heading nodes key, and append subsequent contents in value
        if n in passages_heading_nodes:
            cur_heading = list(n.strings)[0].lower()
            sections[cur_heading] = []
        elif cur_heading is not None:
            sections[cur_heading].append(n)

    return sections

def sections_to_exercises(sections):
    """
    Input:
    -   sections: output of extract_reading_exercise()
    
    Output
    -   {
            'Reading Passage 1': {
                'passage': [
                    '学生: すみません。この本をかりたいです。',
                    'としょかんの人: この学校の学生ですか。',
                ],
                'problems': [
                    {
                        'question': '1. [_______] には何を入れますか。',
                        'options': [' 本は２９日、ざっしは２２日です。',' 本は２２日、ざっしは２９日です。',]
                    },
                ]
            },
        }
    """
    exercises = {}
    for section, contents in sections.items():
        # split reading passage into passage and questions
        if "reading passage" in section or "文章" in section:
            exercises[section] = {"passage": [], "problems": []}
            for content in contents:

                # check if content is problem or text
                problem_pattern = r"\d+\..*"
                if re.match(problem_pattern, list(content.strings)[0]):
                    strs = list(content.strings)
                    qna = {"question" : strs[0], "options": strs[1:]}
                    exercises[section]["problems"].append(qna)
                else:
                    exercises[section]["passage"] += list(content.strings)
        
        # TODO: add other section
    
    return exercises

**usage example**

In [10]:
soup = request_html("https://japanesetest4you.com/japanese-language-proficiency-test-jlpt-n5-reading-exercise-09/")

In [11]:
sections = extract_reading_exercise(soup)
exercises = sections_to_exercises(sections)
exercises

{'reading passage 1': {'passage': ['きのうは友だちのいえへ行って、友だちがつくったりょうりを食べました。すこしからかったです。でも、とてもおいしかったです。わたしも友だちもたくさん食べました。また食べたいです。'],
  'problems': [{'question': '1. ただしいものはどれですか。',
    'options': [' わたしはりょうりをつくりました。',
     ' 友だちはりょうりがじょうずです。',
     ' りょうりはおいしくありませんでした。',
     ' 友だちはあまり食べませんでした。']}]},
 'reading passage 2': {'passage': ['田中さんへ',
   'おとといはどうもありがとう。雨がたくさん降りましたから、田中さんに傘を借りてよかったです。借りた傘はドアのところに置きます。それから昨日作ったお菓子も置きます。どうぞ食べてください。',
   '９月１０日\u3000午後７時\u3000鈴木より'],
  'problems': [{'question': '2. 鈴木さんは９月１０日に何をしましたか。',
    'options': [' お菓子を買いました。', ' 傘を借りました。', ' 傘を返しました。', ' お菓子を作りました。']}]},
 'reading passage 3': {'passage': ['このたてものの中ではしずかにえを見てください。しゃしんをとらないでください。たばこもこまります。たばこは外ですってください。えを見ながら食べたり飲んだりしないでください。'],
  'problems': [{'question': '3. あなたはこのたてものの中で何をしますか。',
    'options': [' えを見ます。', ' たばこをすいます。', ' ごはんを食べます。', ' しゃしんをとります。']}]},
 'reading passage 4': {'passage': ['けさ、山川さんは７時におきました。あさごはんを食べる前にシャワーをあびました。あさごはんを食べながらテレビを見ました。あさごはんを食べたあとで、新聞を読みました。それから会社へ行きました。'],
  'problems

### Split Japanese text into sentences

In [12]:
sentence_tokenizer = SentenceTokenizer()
def split_to_sentences(text):
    sentences = sentence_tokenizer.tokenize(text)
    return sentences

**usage example**

In [13]:
text = "軍人役を任されてまず思ったことは、外見的に強く見せなければならないということでした。ユ・シジンは特殊戦司令部チーム長という役どころですから、まずは筋肉をつけるしかありません。また、強靭な印象を与えるために、ダイエットも並行していました。ロープ降下訓練、アクション訓練、射撃訓練、アルファチームとしての集団戦の訓練などさまざまな訓練を行いました。ナイフを使用して戦うため、剣術訓練を一番熱心にやりましたね。"
split_to_sentences(text)

['軍人役を任されてまず思ったことは、外見的に強く見せなければならないということでした。',
 'ユ・シジンは特殊戦司令部チーム長という役どころですから、まずは筋肉をつけるしかありません。',
 'また、強靭な印象を与えるために、ダイエットも並行していました。',
 'ロープ降下訓練、アクション訓練、射撃訓練、アルファチームとしての集団戦の訓練などさまざまな訓練を行いました。',
 'ナイフを使用して戦うため、剣術訓練を一番熱心にやりましたね。']

### Transform list of exercises to sentence dataset

In [14]:
def reading_data_to_dataframe(reading_data):
    """
    Input:
    - reading_data: list of exercises of each level
        {
            "N1": [
                output of sections_to_exercises() ,
            ],
        }

    Output:
    - dataframe of sentences in each level

    """
    sentences_dataset = []

    for level, exercises in reading_data.items():
        texts = []
        for exercise in exercises:
            for name, reading in exercise.items():
                texts += reading["passage"]
                for problem in reading["problems"]:
                    texts.append(problem["question"])
                    texts += problem["options"]

        # split all texts into sentences
        for text in texts:
            sentences = split_to_sentences(text)
            for sentence in sentences:
                sentences_dataset.append((sentence.strip(), level))

    df = pd.DataFrame(sentences_dataset, columns=["Sentence", "Level"])
    return df

### **Merge everything together**

In [15]:
home_html = request_html(WEBSITE)
links_to_menu = extract_menu_urls(home_html)

readings_data = {}
for i, link in enumerate(links_to_menu):
    menu_html = request_html(link)
    links_to_test = extract_test_urls(menu_html)
    level_data = []
    
    for l in links_to_test:
        test_html = request_html(l)
        sections = extract_reading_exercise(test_html)
        exercises = sections_to_exercises(sections)
        level_data.append(exercises)

    readings_data[f"N{i+1}"] = level_data

In [16]:
readings_data

{'N1': [{'reading passage 1': {'passage': ['人生というものには、いろいろな問題があります。しかし、それらのことも過ぎ去ってみると、あのときに迷わないでやってほんとうによかったな、というような場合が多いのです。そこが大事なところだと思います。ある場合には迷うこともあるでしょう。しかし、しょせん迷ってもお互い自分の知恵裁量というものは、ほんとうは小さいものです。だから、「これはもう仕方がない。ここまでできたのだからこれ以上進んで結果がうまくいかなくても、それは運命だ」と度胸を決めてしまう。そうした場合には、案外、困難だと思っていたことがスムーズにいって、むしろ非常によい結果を生む、ということにもなるのではないかと思うのです。'],
    'problems': [{'question': '1. 筆者がここで最も言いたいことは何か。',
      'options': [' うまくいかない場合は一人で迷うより、みんなの知恵を借りたほうがいい。',
       ' 人生においてしかたないとあきらめることもいい結果をもたらす場合がある。',
       ' 人生は心の持ちようで以外にうまくいく場合もある。',
       ' 難しい問題に遭っても迷わず実践することが大事だ。']}]},
   'reading passage 2': {'passage': ['日本人に個性がないということはよく言われていることだけれど、今世界的に、1 週間、或いは年間にどれだけ働くか、ということについて、常識的な申し合わせが行われていることには、私はいつも',
     '違和感を覚えている',
     '。',
     '私は毎年、身体障害者の方たちとイスラエルやイタリアなどに旅をしているが､一昨年はシナイ山に登った。盲人も６人、ボランティアの助力を得て頂上を究めた。普段、数十歩しか歩けない車椅子の人にも、頂上への道を少しでも歩いてもらった。',
     '障害者にとっての山頂',
     'は、決して現実の山の頂きではない。もし普段100 歩しか歩けない障害者が、頑張ってその日に限り、山道を200 歩歩いて力尽きたら、そここそがその人にとっての光栄ある山頂なのである。',
     '人間が週に何時間働くべきか、ということに

In [17]:
df = reading_data_to_dataframe(readings_data)
df

,Sentence,Level
0,人生というものには、いろいろな問題があります。,N1
1,しかし、それらのことも過ぎ去ってみると、あのときに迷わないでやってほんとうによかったな、とい...,N1
2,そこが大事なところだと思います。,N1
3,ある場合には迷うこともあるでしょう。,N1
4,しかし、しょせん迷ってもお互い自分の知恵裁量というものは、ほんとうは小さいものです。,N1
...,...,...
3438,10. 「４」にはなにをいれますか。,N5
3439,わるい,N5
3440,たのしい,N5
3441,きたない,N5


In [18]:
df.to_csv("jlpt_sentences.csv")